In [21]:
# !nvidia-smi

### 1. Clone the github repo

In [22]:
# !git clone https://github.com/Wangt-CN/DisCo


### 2. Install the package

Ps: Most errors are due to the unsuccessful package installation, please check the installation carefully.


In [23]:
# !pip install --user torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install --user progressbar psutil pymongo simplejson yacs boto3 pyyaml ete3 easydict deprecated future django orderedset python-magic datasets h5py omegaconf einops ipdb
# !pip install --user --exists-action w -r DisCo/requirements.txt
# !pip install git+https://github.com/microsoft/azfuse.git

# # for acceleration
# !pip install --user deepspeed==0.6.3
# !pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers

### 3. Download the pretrained model
Feel free to use our other [checkpoints](https://github.com/Wangt-CN/DisCo#model-checkpoint-google-cloud-tiktok-training-data-fid-fvd-188--more-tiktok-style-training-data-fid-fvd-157) or change to your own model

In [24]:
# !git clone https://huggingface.co/lambdalabs/sd-image-variations-diffusers
# !wget https://storage.googleapis.com/disco-checkpoint-share/checkpoint_ft/moretiktok_nocfg/mp_rank_00_model_states.pt

### 4. Start Running

In [25]:
import os
os.chdir('/mnt/DATA/Personnel/Other learning/Programming/Personal_projects/3_Hackathons_with_buddies/uofthacks2024/model/generate_dance/DisCo/')
#when running this, you have to put the right folder.
os.getcwd()

'/mnt/DATA/Personnel/Other learning/Programming/Personal_projects/3_Hackathons_with_buddies/uofthacks2024/model/generate_dance/DisCo'

In [26]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_ENABLE"] = "0"

# going into this file and importing everything
from DisCo.utils.wutils_ldm import *

from DisCo.agent import Agent_LDM, WarmupLinearLR, WarmupLinearConstantLR
import torch
from DisCo.config.basicargs import BasicArgs #i modifified thid 
from DisCo.utils.lib import *
from DisCo.utils.args import parse_with_cf
from DisCo.utils.dist import dist_init
from DisCo.dataset.tsv_dataset import make_data_sampler, make_batch_data_sampler
from DisCo.finetune_sdm_yaml import get_loader_info, make_data_loader
torch.multiprocessing.set_sharing_strategy('file_system')

/mnt/DATA/Personnel/Other learning/Programming/Personal_projects/3_Hackathons_with_buddies/uofthacks2024/model/generate_dance/DisCo


In [29]:
from DisCo.utils.args import sharedArgs
manual_args = ['--cf', 
'/mnt/DATA/Personnel/Other learning/Programming/Personal_projects/3_Hackathons_with_buddies/uofthacks2024/model/generate_dance/DisCo/config/ref_attn_clip_combine_controlnet/app_demo_image_edit.py', 
'--eval_visu', 'True', '--root_dir', '/content/run_test', '--local_train_batch_size', '32', '--local_eval_batch_size', '32', '--log_dir', 'exp/tiktok_ft', '--epochs', '20', '--deepspeed', '--eval_step', '500',
               '--save_step', '500', '--gradient_accumulate_steps', '1', '--learning_rate', '2e-4', '--fix_dist_seed', 'True', '--loss_target',
               'noise', '--unet_unfreeze_type', 'all', '--guidance_scale', '3', '--refer_sdvae', 'True', '--ref_null_caption', 'False', '--combine_clip_local', 'True', '--combine_use_mask', 'True', '--conds', 'poses','masks', 'model_pretrained.pt', '/content/mp_rank_00_model_states.pt', '/mnt/DATA/Personnel/Other learning/Programming/Personal_projects/3_Hackathons_with_buddies/uofthacks2024/model/generate_dance/pretrained-image-model/model_pretrained.pt', '/content/sd-image-variations-diffusers', '--eval_save_filename', 'try']
parsed_args = sharedArgs.parser.parse_args(args=manual_args)

###### process the args #######
if parsed_args.root_dir:
    BasicArgs.root_dir = parsed_args.root_dir
else:
    parsed_args.root_dir = BasicArgs.root_dir
parsed_args.pretrained_model_path = r"/mnt/DATA/Personnel/Other learning/Programming/Personal_projects/3_Hackathons_with_buddies/uofthacks2024/model/generate_dance/pretrained-image-model/model_pretrained.pt"
#  os.path.join(parsed_args.root_dir, parsed_args.pretrained_model_path)

def parse_with_cf(parsed_args):
    """This function will set args based on the input config file.
    (1) it only overwrites unset parameters,
        i.e., these parameters not set from user command line input
    (2) it also sets configs in the config file but declared in the parser
    """
    # convert to EasyDict object,
    # enabling access from attributes even for nested config
    # e.g., args.train_datasets[0].name
    args = edict(vars(parsed_args))
    if os.path.exists(parsed_args.cf):
        cf = import_filename(parsed_args.cf)
        config_args = edict(vars(cf.Args))
        override_keys = {arg[2:].split("=")[0] for arg in manual_args
                         if arg.startswith("--")}
        # import pdb;pdb.set_trace()
        for k, v in config_args.items():
            if k not in override_keys:
                setattr(args, k, v)
    else:
        raise NotImplementedError('Config filename %s does not exist.' % args.cf)
    return args

args = parse_with_cf(parsed_args)

args.n_gpu = T.cuda.device_count() # local size
args.local_size = args.n_gpu
if args.root_dir not in args.log_dir:
    args.log_dir = os.path.join(args.root_dir, args.log_dir)
if args.stepwise_sample_depth == -1:
    args.interpolation = None
    args.interpolate_mode = None
if args.interpolation != "interpolate":
    args.interpolate_mode = None

assert args.eval_step > 0, "eval_step must be positive"
assert args.save_step > 0, "save_step must be positive"

dist_init(args)
args.dist = args.distributed
args.nodes = args.num_nodes
args.world_size = args.num_gpus
args.train_batch_size = args.local_train_batch_size * args.world_size
args.eval_batch_size = args.local_eval_batch_size * args.world_size
#############################################

cf = import_filename(args.cf)
Net, inner_collect_fn = cf.Net, cf.inner_collect_fn

dataset_cf = import_filename(args.dataset_cf)
BaseDataset = dataset_cf.BaseDataset

# args = update_args(parsed_args, args)

# init models
logger.info('Building models...')
model = Net(args)
print(f"Args: {edict(vars(args))}")

ModuleNotFoundError: No module named 'config.ref_attn_clip_combine_controlnet'

In [30]:
logger.warning("Do eval_visu...")
if getattr(args, 'refer_clip_preprocess', None):
    eval_dataset = BaseDataset(args, args.val_yaml, split='val', preprocesser=model.feature_extractor)
else:
    eval_dataset = BaseDataset(args, args.val_yaml, split='val')
eval_dataloader, eval_info = make_data_loader(
    args, args.local_eval_batch_size,
    eval_dataset)


trainer = Agent_LDM(args=args, model=model)
trainer.eval_demo_pre()

[2024-01-26 23:52:19 <2716821498.py:1> <module>] Do eval_visu...
[2024-01-26 23:52:19 <2716821498.py:1> <module>] Do eval_visu...


NameError: name 'args' is not defined

In [ ]:
def load_image(image):
    if not image.mode == "RGB":
        image = image.convert("RGB")
    return image

@torch.no_grad()
def inference(reference_fg, fg_mask, ref_bg_image, bg_mask, skeleton_img, *args, **kwargs):
    reference_fg = load_image(reference_fg)
    fg_mask = load_image(fg_mask)
    ref_bg_image = load_image(ref_bg_image)
    bg_mask = load_image(bg_mask)
    skeleton_img = load_image(skeleton_img)

    input_data = [reference_fg, fg_mask, ref_bg_image, bg_mask, skeleton_img]
    output_image = trainer.eval_demo_run(input_data, eval_dataset=eval_dataset)
    return output_image

@torch.no_grad()
def inference_masked(reference_fg, ref_bg_image, skeleton_img, *args, **kwargs):
    reference_fg = load_image(reference_fg)
    ref_bg_image = load_image(ref_bg_image)
    skeleton_img = load_image(skeleton_img)

    input_data = [reference_fg, ref_bg_image, skeleton_img]
    output_image = trainer.eval_demo_run_masked(input_data, eval_dataset=eval_dataset)
    return output_image

### 5. Launch the gradio demo

In [ ]:


import gradio as gr
'''
launch app
'''
title = "DisCo Demo (Video Demo Comming Soon!)"
description = """<p style='text-align: center'> <a href='https://disco-dance.github.io/' target='_blank'>Project Page</a> | <a href='https://arxiv.org/pdf/2212.11270.pdf' target='_blank'>Paper</a> | <a href='https://github.com/microsoft/X-Decoder' target='_blank'>Github Repo</a> | <a href='https://youtu.be/wYp6vmyolqE' target='_blank'>Video</a> </p>
<p>Skip the queue by duplicating this space and upgrading to GPU in settings</p>
<a href="https://huggingface.co/spaces/xdecoder/Demo?duplicate=true"><img src="https://bit.ly/3gLdBN6" alt="Duplicate Space"></a>
"""



with gr.Blocks() as demo:
    gr.Markdown(
    """
    # DisCo Demo (Video Demo Comming Soon!)
    Start edit the human with provided human foreground, background, pose.

    Note that for self-uploaded images, TikTok-Style human images are preferred.

    [Project Page](https://disco-dance.github.io/) | [Github](https://github.com/Wangt-CN/DisCo)
    """)

    with gr.Row().style(equal_height=False):
        with gr.Column(min_width=400, scale=2):
            input_fg = gr.Image(type='pil',label="Foreground Image")
            gr.Examples(examples=["./demo_data/fg/masked_images/00035.png", "./demo_data/fg/masked_images/00335.png", "./demo_data/fg/masked_images/00147.png", "./demo_data/fg/masked_images/00072.png", "./demo_data/fg/masked_images/00115.png"], inputs=input_fg)

            input_bg = gr.Image(type='pil',label="Background Image")
            gr.Examples(examples=["./demo_data/bg/masked_images/00035.png", "./demo_data/bg/masked_images/00335.png", "./demo_data/bg/masked_images/00147.png", "./demo_data/bg/masked_images/00072.png", "./demo_data/bg/masked_images/00115.png"], inputs=input_bg)

            input_pose = gr.Image(type='pil',label="Target Pose",scale=1)
            gr.Examples(examples=["./demo_data/pose_img/0049.png","./demo_data/pose_img/0198.png","./demo_data/pose_img/0213.png","./demo_data/pose_img/0264.png","./demo_data/pose_img/0144.png","./demo_data/pose_img/0054.png"], inputs=input_pose)

            btn = gr.Button("Generate")


        with gr.Column(min_width=150):
            output_img = gr.Image(type='pil',label="Edited Human Image")

    btn.click(inference_masked, inputs=[input_fg, input_bg, input_pose], outputs=[output_img])

demo.queue(concurrency_count=2)
demo.launch(share=True)

<ipython-input-7-11e6eef69ecf>:31: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  with gr.Row().style(equal_height=False):


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e249d08ffc1f80b5e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
